In [1]:
import numpy as np
import pandas as pd

In [2]:
interactions_path = "./data/RAW_interactions.csv"
recipes_path = "./data/RAW_recipes.csv"

### Loading Data

In [3]:
interactions_raw = pd.read_csv(interactions_path)
recipes_raw = pd.read_csv(recipes_path)

In [4]:
interactions_raw.head(1)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...


In [45]:
recipes_raw.head(1)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7


In [5]:
interactions_raw.groupby('recipe_id').head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [6]:
interactions_raw.groupby(['recipe_id', 'rating'])['user_id'].agg('count').reset_index()

,recipe_id,rating,user_id
0,38,4,3
1,38,5,1
2,39,3,1
3,40,1,1
4,40,4,2
...,...,...,...
367682,537459,5,1
367683,537485,5,1
367684,537543,0,1
367685,537671,0,1


In [31]:
K = 6
def count_ratings(x):
    ret = [0] * K
    ratings = list(x.rating)
    counts = list(x.user_id)
    for rating, count in zip(ratings, counts):
        ret[rating] = count
    return ret

In [34]:
recipes_with_rating_counts = interactions_raw.groupby(['recipe_id', 'rating'])['user_id'].agg('count').reset_index().groupby('recipe_id')[['rating', 'user_id']].apply(count_ratings).reset_index(name='ratings')

In [36]:
import math
def starsort(ns):
    """
    http://www.evanmiller.org/ranking-items-with-star-ratings.html
    """
    N = sum(ns)
    K = len(ns)
    s = list(range(K,0,-1))
    s2 = [sk**2 for sk in s]
    z = 1.65
    def f(s, ns):
        N = sum(ns)
        K = len(ns)
        return sum(sk*(nk+1) for sk, nk in zip(s,ns)) / (N+K)
    fsns = f(s, ns)
    return fsns - z*math.sqrt((f(s2, ns)- fsns**2)/(N+K+1))

In [38]:
recipes_with_rating_counts['popularity'] = recipes_with_rating_counts['ratings'].apply(starsort)

In [39]:
recipes_with_rating_counts.head()

,recipe_id,ratings,popularity
0,38,"[0, 0, 0, 0, 3, 1]",2.004010
1,39,"[0, 0, 0, 1, 0, 0]",2.500563
2,40,"[0, 1, 0, 0, 2, 6]",1.696731
3,41,"[0, 0, 0, 0, 1, 1]",2.047372
4,43,"[0, 1, 0, 0, 0, 0]",2.742411


In [46]:
top_k_recipes = list(recipes_with_rating_counts.sort_values('popularity', ascending=False).head(10).recipe_id)

In [49]:
top_k_recipes

[18260, 78524, 517511, 507069, 534955, 349190, 28221, 187125, 89973, 154248]

In [48]:
recipes_raw.set_index('id').loc[top_k_recipes].reset_index(inplace=False)

,id,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,18260,sakura sauce,9,29450,2002-01-27,"['15-minutes-or-less', 'time-to-make', 'course...","[2001.2, 342.0, 2.0, 126.0, 11.0, 148.0, 0.0]",9,"['in a medium bowl , add the 3 egg yolks , who...",smooth japanese creamy sauce for seafood or ch...,"['egg yolks', 'egg', 'vegetable oil', 'white p...",7
1,78524,vegan marshmallows,50,111711,2003-12-10,"['60-minutes-or-less', 'time-to-make', 'course...","[117.7, 0.0, 89.0, 0.0, 0.0, 0.0, 10.0]",13,['soak agar-agar in cup cold water until water...,most marshmallows are neither vegan or vegetar...,"['agar-agar', 'cold water', 'granulated sugar'...",7
2,517511,2 minute microwave brownie in a mug,4,1802973346,2014-07-29,"['15-minutes-or-less', 'time-to-make', 'course...","[370.7, 44.0, 51.0, 6.0, 7.0, 22.0, 10.0]",10,"['add 2 tablespoons of butter or oil', 'melt b...",this is just a simple 2 min. brownie in a mug ...,"['oil', 'water', 'flour', 'salt', 'sugar', 'co...",7
3,507069,jersey disco fries,25,637765,2013-09-18,"['30-minutes-or-less', 'time-to-make', 'course...","[555.6, 44.0, 28.0, 216.0, 40.0, 81.0, 17.0]",15,"['pre-heat oven to 450 degrees f', 'arrange st...",a jersey diner staple -- great for when you're...,"['steak-style french fries', 'cheez whiz', 'gr...",5
4,534955,tunnel of fudge banana cake,65,33186,2018-01-18,"['time-to-make', 'course', 'main-ingredient', ...","[602.0, 58.0, 162.0, 10.0, 15.0, 95.0, 23.0]",32,"['to make chocolate banana cake:', 'in a bowl ...",an irresistible reimagining of a fudge-filled ...,"['butter', 'sugar', 'dark brown sugar', 'salt'...",17
5,349190,gluten free breaded chicken,50,1123903,2009-01-11,"['60-minutes-or-less', 'time-to-make', 'main-i...","[250.3, 12.0, 0.0, 15.0, 57.0, 13.0, 4.0]",5,['combine all dry ingredients into a large bow...,this is a breaded chicken that is made with oa...,"['boneless skinless chicken thighs', 'ground o...",11
6,28221,pecan pie jam,35,10404,2002-05-13,"['60-minutes-or-less', 'time-to-make', 'course...","[2408.0, 222.0, 1089.0, 14.0, 26.0, 188.0, 96.0]",6,['in a large stainless steal or enamel pot mel...,this jam is unbelieveable on toasted english m...,"['pecans', 'white sugar', 'dark brown sugar', ...",8
7,187125,homemade whipped topping substitute or dream ...,20,344087,2006-09-20,"['30-minutes-or-less', 'time-to-make', 'prepar...","[152.6, 8.0, 72.0, 4.0, 11.0, 17.0, 6.0]",5,"['if kitchen is warm , prechill a deep , narro...",handy to have in emergency recipe that uses ev...,"['cold water', 'powdered milk', 'salt', 'lemon...",6
8,89973,marie callender s banana cream pie by todd wilbur,125,59532,2004-04-25,"['time-to-make', 'course', 'main-ingredient', ...","[965.8, 110.0, 122.0, 24.0, 21.0, 175.0, 24.0]",24,['beat together the butter and shortening unti...,this recipe is from top secret restaurant reci...,"['butter', 'shortening', 'flour', 'sugar', 'sa...",15
9,154248,teething biscuits for baby,20,108846,2006-02-01,"['30-minutes-or-less', 'time-to-make', 'course...","[15.7, 0.0, 4.0, 0.0, 1.0, 0.0, 1.0]",15,['preheat the oven to 350f and use ungreased c...,good and pure ingredients are used in these ba...,"['egg', 'honey', 'sugar', 'salt', 'soy flour',...",8
